In [8]:
import torch
from torch.autograd import Variable

import pandas as pd
import numpy as np
import sys
import h5py
import pickle
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import StandardScaler
from sklearn.grid_search import GridSearchCV
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from scipy.sparse import csr_matrix, hstack
from sklearn.cross_validation import KFold, train_test_split
import matplotlib.patches as mpatches
import time

%matplotlib inline

In [9]:
train = pd.read_csv('train.csv')

cat_names = [c for c in train.columns if 'cat' in c]

train = pd.get_dummies(data=train, columns=cat_names)

features = [x for x in train.columns if x not in ['id','loss']]

train_x = np.array(train[features])

ntrain = train_x.shape[0]

# np.log(train['loss'] + 200) provides
# a better score, but let's keep it simple now
train_y = np.array(train['loss'])

print (train_x.shape)
print (train_y.shape)
train_x.shape, train_y.shape

(188318, 1153)
(188318,)


((188318, 1153), (188318,))

In [14]:
# N is batch size; D_in is input dimension;
# H is hidden dimension; D_out is output dimension.
N = train_x.shape[0]
D = [train_x.shape[1],351,293,46]
dtype = torch.cuda.FloatTensor # Uncomment this to run on GPU
# Create random Tensors to hold inputs and outputs, and wrap them in Variables.
x = Variable(torch.from_numpy(train_x).type(dtype))
y = Variable(torch.from_numpy(train_y).type(dtype), requires_grad=False)

# Use the nn package to define our model and loss function.
model = torch.nn.Sequential(
    torch.nn.Linear(D[0], D[1]),
    torch.nn.BatchNorm1d(D[1]),    
    torch.nn.ReLU(),
    torch.nn.Dropout(),
    torch.nn.Linear(D[1], D[2]),
    torch.nn.BatchNorm1d(D[2]),
    torch.nn.ReLU(),
    torch.nn.Dropout(),
    torch.nn.Linear(D[2], D[3]),
    torch.nn.BatchNorm1d(D[3]),
    torch.nn.Dropout(),
).type(dtype)
loss_fn = torch.nn.MSELoss(size_average=False).type(dtype)
# Use the optim package to define an Optimizer that will update the weights of
# the model for us. Here we will use Adam; the optim package contains many other
# optimization algoriths. The first argument to the Adam constructor tells the
# optimizer which Variables it should update.
learning_rate = 1e-4
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
for t in range(500):
    # Forward pass: compute predicted y by passing x to the model.
    y_pred = model(x)

    # Compute and print loss.
    loss = loss_fn(y_pred, y)
    print(t, loss.data[0])

    # Before the backward pass, use the optimizer object to zero all of the
    # gradients for the variables it will update (which are the learnable weights
    # of the model)
    optimizer.zero_grad()

    # Backward pass: compute gradient of the loss with respect to model
    # parameters
    loss.backward()

    # Calling the step function on an Optimizer makes an update to its
    # parameters
    optimizer.step()

RuntimeError: CUDNN_STATUS_NOT_SUPPORTED. This error may appear if you passed in a non-contiguous input.